# Association by Rule implementation project in Big Data analysis course
## Amitai Stern and Eran Katz

In [1]:
sc

<div>
            <p><b>SparkContext</b></p>

            <p><a href="http://DESKTOP-OCHVAKQ:4041">Spark UI</a></p>

            <dl>
              <dt>Version</dt>
                <dd><code>v2.3.3</code></dd>
              <dt>Master</dt>
                <dd><code>local[*]</code></dd>
              <dt>AppName</dt>
                <dd><code>PySparkShell</code></dd>
            </dl>
        </div>

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# get dataset:

In [3]:
PATH = "/home/spark-vm/dataSets/"
user_searches_rdd = sc.textFile("%suser-ct-test-collection-01.txt" % PATH)

In [4]:
#split by tab
user_searches_rdd = user_searches_rdd.map( lambda line: line.split('\t'))
#remove header
header = user_searches_rdd.first()
user_searches_rdd = user_searches_rdd.filter( lambda line: line != header)

## remove unwanted columns:

In [5]:
#remove all but first two elements from each row - remain with AnonID and Query in each row
user_query_rdd = user_searches_rdd.map( lambda line: (line[0],line[1]))

In [6]:
#same number of partitions
user_searches_rdd.getNumPartitions(), user_query_rdd.getNumPartitions()

(7, 7)

# create inverted index: userId - list[ (q1,1), (q2, 1) , ... , (qn, 1)]
## Also:
## -remove url elements from queries
## -trim the queries 
## -sort the resulting list and remove duplicates
## -filter out empty lists

In [7]:
# more efficient than reduce by key with the lambda function ( lambda x,y: x+y) 
# extend() doesn't copy the list every time, so it is not n^2 in complexity to length of the list...
def addToList(x,y):
    x.extend(y)
    return x

def pairKV(lst): #all terms are preprocessed for reduce by key --> (term, 1)
    newLst = list()
    for item in lst:
        item = tuple((item, 1))
        newLst.append(item)
    return newLst

def preprocess(lst):
    newLst = list()
    for item in lst:
        item = item.strip()
        item = item.replace('www.','')
        item = item.replace('.com','')
        item = item.replace('.html','')
        if len(item) > 1:
            newLst.append(item)
    return newLst
    

user_query_invertedIndex = user_query_rdd.map( lambda x: (x[0], [x[1]]))\
                                         .reduceByKey( lambda x,y: addToList(x,y))\
                                         .map( lambda kv: (kv[0],sorted(set(preprocess(kv[1])))))\
                                         .filter( lambda kv: len(kv[1])>0)\
                                         .map( lambda kv: (kv[0],pairKV(kv[1]))) #make each query: (query,1)


# used set() on each list instead of distinct() on all of the rdd to avoid a massive shuffle
# result:
# inverted index from user to list of sorted distinct queries, and list of all pair sets (sorted too): 
# example -((u'5562375',[(u"barrett's esophogus", 1),(u'chase.com', 1),(u'e coli', 1), ... , (u'zillow', 1)]

# calculate minsup according to 
#### http://data-mining.philippe-fournier-viger.com/how-to-auto-adjust-the-minimum-support-threshold-according-to-the-data-size/

## minsup(65516)=e^(-0.00004*65516 -10) + 0.0001 =~ 0.0001
## which means that any term with less than 6 occurences is bellow the minsup

In [8]:
num_transactions = user_query_invertedIndex.count()

In [9]:
import math
minsup = exp(-0.00004*num_transactions -10) + 0.0001
r_minsup = floor(num_transactions*minsup)
print "minsup is: %f, relative minsup is: %f" %(minsup, r_minsup)

minsup is: 0.000103, relative minsup is: 6.000000


# Count occurences of each query in the dataset, and remove queries with support smaller than minsupport

In [10]:
#count occurences of each term
term_occurences_RDD = user_query_invertedIndex.flatMap( lambda row: row[1])\
                                                .reduceByKey( lambda x,y: x+y)\
                                                .filter( lambda term: term[1] >= r_minsup and len(term[0])>2)
#(u'sec 8 apt for rent baltimore md', 1),
#(u'new york hurricane', 1),
#(u'free stephani wells sex clip', 1),
#(u'hennings', 1),
#(u'canes', 5),
#(u'ricediet.com', 1),
#(u'little my maid pc game', 1)

# Create list of pairs from the user-queries, and reduce by key
## creating the list from the users queries prevents looking for rules of  a pair that doesn't exist in the dataset.

In [11]:
def allPairsList(lst):#all distinct pairs possible from list of terms (as sets) and prepared for reduce by key
    newLst = list()
    for i in range(0,len(lst)):
        for j in range(i+1, len(lst)):
                item = tuple((lst[i] + '\t' + lst[j], 1)) # \t as delimiter between queries
                newLst.append(item)
    return newLst

def removeNumOccur(lst): # get rid of the comma '1' after each query
    newLst = list()
    for item in lst:
        newLst.append(item[0])
    return newLst

pair_occurences_RDD = user_query_invertedIndex.map( lambda kv: (kv[0], removeNumOccur(kv[1])))\
                                              .filter( lambda kv: len(kv[1]) > 1)\
                                              .map( lambda kv: (allPairsList(kv[1])))\
                                              .flatMap( lambda row: row)\
                                              .reduceByKey( lambda x,y: x+y)\
                                              .filter( lambda pair: int(pair[1]) > 1)
#this is the bottle neck, big shuffle...

In [12]:
# we now have two rdd's one with (query, # of occurrences) the other - (query1\tquery2, # of occurrences)
# Next we shall create tuples (X, Y, supp(XUY), supp(X)) for every possible pair:
# split pair list: (query1\tquery2, # of occurrences) --> (query1, query2, # of occurrences)

In [13]:
# for each pair (q1, q2, # q1Uq2) add (q2, q1, # q1Uq2) to the RDD:
pair_occurences_RDD = pair_occurences_RDD.map( lambda kv: (kv[0].split('\t')[0], kv[0].split('\t')[1], kv[1]))\
                                         .map( lambda arr: (((arr[0], (arr[1],arr[2]))), (arr[1], (arr[0],arr[2]))))\
                                         .flatMap( lambda ab: ab)

# Join the two RDDs anf filter out value containing None

In [14]:
# left join the term_occurences_RDD on to this RDD, according to the first term:
pair_term_supp_RDD = pair_occurences_RDD.join(term_occurences_RDD)

In [15]:
#filter out None values:
pair_term_supp_RDD = pair_term_supp_RDD.filter( lambda x: x[1][0])

# For each line calculate th confidence, resulting in: 
# X | Y | Confidence of rule: X-->Y | # of pairs XUY in dataset

In [16]:
def intoTup(t): #into a four value tuple
    key = t[0]
    value = t[1]
    lst = list()
    lst.append(key)
    lst.extend(list(value[0]))
    lst.append(value[1])
    res = tuple(lst)
    return res
    
def calcConf(line):
    conf = line[2] / float(line[3])
    lst = list(line)
    numPairs = line[2]
    lst = lst[:2]
    lst.append(conf)
    lst.append(numPairs)
    res = tuple(lst)
    return res


In [17]:
conf_RDD = pair_term_supp_RDD.map(lambda line: intoTup(line))\
                             .map(lambda line: calcConf(line))\
                             .filter( lambda rule: float(rule[2])>0.3) # lowest confidence to be considered

# Pickling the RDD:

In [18]:
conf_RDD.saveAsPickleFile('assoc_rules_pkl')

## Or: save the resulting RDD as a csv file by removing the following lines out of comment

In [18]:
# save RDD result:
#def toCSVLine(data):
#  return ','.join(str(d).encode('utf-8') for d in data)

#lines = conf_RDD.map(toCSVLine)
#lines.saveAsTextFile('result_assoc_rules.txt')

# ------------------------------------------------

# Analysing the results:

In [19]:
# for interesting results it is prefferable to ignore pairs containing term like google - most chances are that 
# any serch term appears every time in a transaction with google or yahoo... 
# given a term x, x-->'google' would probably be higher than 0.6 
most_common_terms_lst = term_occurences_RDD.filter( lambda x: int(x[1]) >= 3500).map( lambda x: x[0]).collect()
most_common_terms_lst

[u'yahoo', u'mapquest', u'ebay', u'myspace', u'google']

# filter results where 'Y' is from most_common_terms_lst

In [20]:
conf_RDD_analyse = conf_RDD.filter( lambda x: x[1] not in most_common_terms_lst)

In [21]:
#create data frame
from pyspark.sql.types import *
fields = [StructField("X", StringType(), True),\
          StructField("Y", StringType(), True),\
          StructField("Confidence", StringType(), True),\
          StructField("Amount of Pairs", StringType(), True)]
schema = StructType(fields)
AssociationRule_res_df = spark.createDataFrame(conf_RDD_analyse, schema)

# 4b. Look for 3 interesting relations found in the date set

In [22]:
AssociationRule_res_df.filter("Confidence > 0.6").orderBy("Confidence",ascending=False).show(500)

+--------------------+--------------------+------------------+---------------+
|                   X|                   Y|        Confidence|Amount of Pairs|
+--------------------+--------------------+------------------+---------------+
|     browsersettings|    browser settings|0.9166666666666666|             11|
|              imhelp|             im help|0.8888888888888888|              8|
|          imsettings|         im settings|0.8888888888888888|              8|
|                 poo|                pogo|0.8888888888888888|              8|
|              hotmal|             hotmail|             0.875|              7|
|             any who|              anywho|             0.875|              7|
|             viewpix|              vzwpix|             0.875|              7|
|         localhookup|        localhookupz|             0.875|              7|
|                ikia|                ikea|             0.875|              7|
| accountcenteronline|accountcentralonline|0.8571428

In [23]:
AssociationRule_res_df.filter("Confidence > 0.6").filter("Y == 'american idol'").orderBy("Confidence",ascending=False).show(10)

+------------------+-------------+------------------+---------------+
|                 X|            Y|        Confidence|Amount of Pairs|
+------------------+-------------+------------------+---------------+
|         dial idol|american idol|0.8333333333333334|              5|
|american idol tour|american idol|0.7647058823529411|             13|
|      elliot yamin|american idol|              0.75|              6|
|      ameican idol|american idol|0.6666666666666666|              4|
+------------------+-------------+------------------+---------------+



# +------------------+------------------+------------------+---------------------+
# |                 X     |            Y          |    Confidence|Amount of Pairs|
# +------------------+------------------+------------------+---------------------+
# |     elliot yamin|american idol|              0.75    |                          6|


### Efraym Elliott Yamin (born July 20, 1978) is an American singer known for his  hit single "Wait for You" and      placing third on the fifth season of American Idol.

## https://en.wikipedia.org/wiki/Elliott_Yamin

In [24]:
AssociationRule_res_df.filter("Confidence > 0.6").filter("Y == 'honda'").orderBy("Confidence",ascending=False).show(10)

+-----+-----+------------------+---------------+
|    X|    Y|        Confidence|Amount of Pairs|
+-----+-----+------------------+---------------+
|isuzu|honda|0.6666666666666666|              4|
+-----+-----+------------------+---------------+



# +-------+--------+-------------------------+----------------------+
# |     X   |         Y|        Confidence      |Amount of Pairs|
# +-------+--------+-------------------------+----------------------+
# | isuzu|honda|0.6666666666666666|            4              |

##  Isuzu is a Japanese commercial vehicles and diesel engine manufacturing company headquartered in Tokyo.
#### https://en.wikipedia.org/wiki/Isuzu_Motors

## Honda is a Japanese public multinational conglomerate corporation primarily known as a manufacturer of automobiles, aircraft, motorcycles, and power equipment.
#### https://en.wikipedia.org/wiki/Honda

In [25]:
AssociationRule_res_df.filter("Confidence > 0.6").filter("Y == 'fall out boy'").orderBy("Confidence",ascending=False).show(10)

+----------+------------+------------------+---------------+
|         X|           Y|        Confidence|Amount of Pairs|
+----------+------------+------------------+---------------+
|pete wentz|fall out boy|0.6666666666666666|              4|
+----------+------------+------------------+---------------+



# +------------------------+---------+----------------------------+-------------------------+
# |     X                         |          Y|         Confidence         |Amount of Pairs     |
# +------------------------+---------+-----------------------------+-------------------------+
# |          pete wentz|        fall out boy|0.6666666666666666|              4|

## Peter Lewis Kingston Wentz III, is an American musician. He is best known for being the bassist, lyricist, and backing vocalist for the American rock band Fall Out Boy.

https://en.wikipedia.org/wiki/Pete_Wentz



In [26]:
AssociationRule_res_df.filter("Confidence > 0.8").filter("Y == 'walmart'").orderBy("Confidence",ascending=False).show(10)

+---------------+-------+------------------+---------------+
|              X|      Y|        Confidence|Amount of Pairs|
+---------------+-------+------------------+---------------+
|sinus infection|walmart|0.8571428571428571|              6|
+---------------+-------+------------------+---------------+



# +------------------------+---------+----------------------------+-------------------------+
# |     X                         |          Y|         Confidence         |Amount of Pairs     |
# +------------------------+---------+-----------------------------+-------------------------+
# |     sinus infection|             walmart|0.8571428571428571|              6|

# They sell everything at walmart: 
https://www.walmart.com/c/kp/sinus-medicines

# Almost all of the rules with confidence 0.8 or higher are a spelling error
# such as: 
# |             my spac -->  my space               | 0.8571428571428571
# |      carttonnetwork --> cartoonnetwork| 0.8333333333333334
# |              kholes --> kohls                         | 0.8333333333333334
# |           dicionary --> dictionary               | 0.8333333333333334 